# §8 Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchical Indexing
* Reordering and Sorting Levels
* Summary Statistics by Level
* Indexing with a DataFrame’s columns

## 8.2 Combining and Merging Datasets
* Database-Style DataFrame Joins
* Merging on Index
* Concatenating Along an Axis
* Combining Data with Overlap

## 8.3 Reshaping and Pivoting
* Reshaping with Hierarchical Indexing
* Pivoting “Long” to “Wide” Format
* Pivoting “Wide” to “Long” Format

## 8.4 Conclusion

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## § 8.1 Hierarchical Indexing

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [5]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [9]:
# Selection is even possible from an “inner” level
data.loc[:,2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [10]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [11]:
type(data)

pandas.core.series.Series

In [12]:
# rearrange the data into a DataFrame using its unstack method:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [13]:
type(data.unstack())

pandas.core.frame.DataFrame

In [14]:
data.unstack().columns

Int64Index([1, 2, 3], dtype='int64')

In [15]:
data.unstack().index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [16]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [17]:
# With a DataFrame, either axis can have a hierarchical index:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [18]:
frame.columns

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           )

In [19]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [20]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [21]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [32]:
# expected error
# frame['a']

In [28]:
frame.loc['a']

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5

### 可建立並重複使用 MultiIndex

```python
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
```

### § 8.1.1 Reordering and Sorting Levels

In [ ]:
frame

In [ ]:
frame.swaplevel('key1', 'key2')

In [ ]:
frame.sort_index(level=0)

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

### 8.1.2 Summary Statistics by Level

In [ ]:
frame

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color', axis=1)

### 8.1.3 Indexing with a DataFrame's columns

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
frame2 = frame.set_index(['c', 'd'])
frame2

In [ ]:
frame.set_index(['c', 'd'], drop=False) # 保留原本會被 drop的欄位

In [ ]:
frame2

In [ ]:
frame2.reset_index()

## 8.2 Combining and Merging Datasets

### § 8.2.1 Database-Style DataFrame Joins

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2)  # 預設為交集(inner)，保留 df1與 df2相同的 key

In [ ]:
pd.merge(df1, df2, on='key') # 指定 key

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # key 的名字不一樣 (即無相同 key名)，可分別指定

In [ ]:
df3

In [ ]:
df4

In [ ]:
pd.merge(df1, df2, how='outer')  # 聯集

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on='key', how='left')  # 左方表格所有 key

In [ ]:
pd.merge(df1, df2, how='inner')  # 多對多 join會產生列的笛卡兒積

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')  # 多個 key進行合併

In [ ]:
left

In [ ]:
right

In [ ]:
pd.merge(left, right, on='key1')
# 合併表格中，不為 key的欄位名字相同時，可利用 suffixes加上後綴詞
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) 

### 8.2.2 Merging on Index

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

In [ ]:
right1

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)  # 使用 index進行 merge

In [ ]:
# pd.merge(left1, right1, left_on='key')  # 使用 index進行 merge

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

In [ ]:
righth

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)  # 多階層 index進行合併


In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)  # 兩邊同時使用 index進行合併

In [ ]:
left2.join(right2, how='outer')  # join : 預設利用 index進行合併

In [ ]:
left1

In [ ]:
right1

In [ ]:
left1.join(right1, on='key')  # left1 以 key作為合併欄位，right1 則用其 index

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

In [ ]:
right2

In [ ]:
left2

In [ ]:
left2.join([right2, another]) 

In [ ]:
left2.join([right2, another], how='outer')

### § 8.2.3 Concatenating Along an Axis

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

In [ ]:
np.concatenate([arr, arr], axis=1)  # axis=0 : 橫軸操作 , axis=1 : 縱軸操作 

In [ ]:
np.concatenate([arr, arr], axis=0)  # axis=0 : 橫軸操作 , axis=1 : 縱軸操作 

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
s1

In [ ]:
s2

In [ ]:
s3

In [ ]:
pd.concat([s1, s2, s3], sort = True)

In [ ]:
pd.concat([s1, s2, s3], axis=1, sort = True)  # axis=0 : 橫軸操作 , axis=1 : 縱軸操作 

In [ ]:
s4 = pd.concat([s1, s3], sort = True)
s4

In [ ]:
pd.concat([s1, s4], axis=1, sort = True)  # axis=0 : 橫軸操作 , axis=1 : 縱軸操作 

In [ ]:
pd.concat([s1, s4], axis=1, join='inner', sort = True)

In [ ]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']], sort = True)  # 自定 index軸

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'], sort = True)  # one:s1,  two:s1,  three:s3
result

In [ ]:
result.unstack()

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'], sort = True)

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort = True)

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort = True)

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'], sort = True)

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], ignore_index=True, sort = True) # 若資料完全不相關，可以不保留原 index進行合併

### § 8.2.4 Combining Data with Overlap

#### 有些資料無法用 merge 或 concat 合併

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[1] = 'nan'
a

In [ ]:
b

In [ ]:
np.where(pd.isnull(a), b, a)  # 條件成立回傳 b的元素，不成利回傳 a的元素

In [ ]:
b.combine_first(a) # 以 b 為主

In [ ]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

## § 8.3 Reshaping and Pivoting

### § 8.3.1 Reshaping with Hierarchical Indexing

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack() # stack預設忽略 nan

In [ ]:
data2.unstack().stack(dropna=False)

In [ ]:
result

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

In [ ]:
df.unstack('number')

In [ ]:
df.unstack('state').stack('side')

### § 8.3.2 Pivoting “Long” to “Wide” Format

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value') # index, header, value
pivoted

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item')   # 拆開 item
pivoted[:5]

In [ ]:
pivoted['value'][:5]

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')  # 同樣獲得拆開 item
unstacked[:7]

### § 8.3.3 Pivoting “Wide” to “Long” Format

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])  # 以 key進行分組
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index()

In [ ]:
reshaped.reset_index().columns.name

In [ ]:
df

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])

In [ ]:
pd.melt(df, value_vars=['key', 'A', 'B'])

## § 8.4 Conclusion

## Learn To Merge and Join DataFrames Easily with Pandas
https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/